In [23]:
import numpy as np
#in case we need to repeat experiment
#np.random.seed(255)

import pandas as pd
pd.options.display.max_rows = 22

import matplotlib.pyplot as plt
plt.style.use('classic')

import seaborn as sns
sns.set()

#sklearn imports
from sklearn.cluster import KMeans
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from pandas.api.types import CategoricalDtype
from sklearn.naive_bayes import GaussianNB
import pickle # allows for model to be saved/load to file
import time

#Use print instead of display when run as python script
pyscript = True

#Classifier verborsity where supported
verbose_level=3

#Multiclass classification, binary if falase
multiclass = False
over_sample = True

#inputfile = 'CKME136X10_2018_Data_CTF.csv'
if multiclass:
    inputfile_train_O = 'CKME136X10_2018_Data_CTFB_M_O_Train.csv'
    inputfile_train_U = 'CKME136X10_2018_Data_CTFB_M_U_Train.csv'
    inputfile_test = 'CKME136X10_2018_Data_CTFB_M_Test.csv'
else:
    inputfile_train_O = 'CKME136X10_2018_Data_CTFB_B_O_Train.csv'
    inputfile_train_U = 'CKME136X10_2018_Data_CTFB_B_U_Train.csv'
    inputfile_test = 'CKME136X10_2018_Data_CTFB_B_Test.csv'

if over_sample:
    datafile_train = inputfile_train_O
else:
    datafile_train = inputfile_train_U

datafile_test = inputfile_test
    
model_max_iter = 1000
datestr = 'dec_05_binary_run_1000_KBO'

#Model Store
file_lr = 'lr_' + datestr + '.model'
file_lr_l1 = 'lr_l2_' + datestr + '.model'
file_dt = 'dt_' + datestr + '.model'
file_svm = 'svm_' + datestr + '.model'
file_knn = 'knn_' + datestr + '.model'
file_mlp = 'mlp_' + datestr + '.model'
file_kmean = 'kmean_' + datestr + '.model'
file_nbayes = 'nbayes_' + datestr + '.model'

file_final_train = 'final_train_' + datestr + '.csv'
file_final_test = 'final_test_' + datestr + '.csv'

#Enable Optimization Algorithms
enable_grid_search = False
svm_c = 1
svm_gamma = 1
nfold = 10

enable_lr_l1 = True
predict_lr_l1 = True

# Enable Algorithms
enable_lr = True
enable_dt = True
enable_svm = True
enable_knn = True
enable_mlp = True
enable_nbayes = True

predict_lr = False
predict_dt = True
predict_svm = True
predict_knn = True
predict_mlp = True
predict_nbayes = True

C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#print("Sample size: {}".format(sampleN))

if multiclass:
    print("Multi-Class Classification: Enabled")
else:
    print("Multi-Class Classification: Disabled")

if enable_lr:
    print("Logistic Regression: Enabled")
else:
    print("Logistic Regression: Disabled")
    
if enable_dt:
    print("Decision Tree: Enabled")
else:
    print("Decision Tree: Disabled")
    
if enable_svm:
    print("Support Vector Machines: Enabled")
else:
    print("Support Vector Machines: Disabled")

if  enable_knn:
    print("KNN: Enabled")
else:
    print("KNN: Disabled")
    
if enable_nbayes:
    print("Naive Bayes: Enabled")
else:
    print("Naive Bayes: Disabled")
    
if enable_mlp:
    print("MLP: Enabled")
else:
    print("MLP: Disabled")

Multi-Class Classification: Disabled
Logistic Regression: Enabled
Decision Tree: Enabled
Support Vector Machines: Enabled
KNN: Enabled
Naive Bayes: Enabled
MLP: Enabled


In [3]:
#load data
df_test = pd.read_csv(datafile_test, engine = 'python')
df_train = pd.read_csv(datafile_train, engine = 'python')
df = df_train.copy()

print(df_test.head(2))
print(df_train.head(2))

   C_YEAR  C_MNTH  C_WDAY  C_HOUR  C_VEHS  C_CONF  C_RCFG  C_WTHR  C_RSUR  \
0       1       5       5       2       3      21       2       1       1   
1      14       2       6       1       4      24       1       1       1   

   C_RALN  C_TRAF  V_TYPE  V_YEAR  P_SEX  P_AGE  P_PSN  P_SAFE  P_USER  P_ISEV  
0       1       1       6       2      1     34      1       2       1       0  
1       2       7       1       4      0     47      3       2       2       1  
   C_YEAR  C_MNTH  C_WDAY  C_HOUR  C_VEHS  C_CONF  C_RCFG  C_WTHR  C_RSUR  \
0       3       5       4       3       2      51       1       1       1   
1      16       3       1       1       2      51       1       1       1   

   C_RALN  C_TRAF  V_TYPE  V_YEAR  P_SEX  P_AGE  P_PSN  P_SAFE  P_USER  P_ISEV  
0       1       7       1       3      1     18      1       2       2       1  
1       1       7       6       4      1     19      1       2       1       1  


In [4]:
df_test_cat = df_test.astype('category').copy()
df_train_cat = df_train.astype('category').copy()

In [ ]:
# convert to the correct type
df_train_cat['C_YEAR'] = df_train_cat['C_YEAR'].astype(CategoricalDtype(ordered=True))
df_train_cat['C_MNTH'] = df_train_cat['C_MNTH'].astype(CategoricalDtype(ordered=True))
df_train_cat['C_WDAY'] = df_train_cat['C_WDAY'].astype(CategoricalDtype(ordered=True))
df_train_cat['C_HOUR'] = df_train_cat['C_HOUR'].astype(CategoricalDtype(ordered=True))
df_train_cat['C_VEHS'] = df_train_cat['C_VEHS'].astype('int')
df_train_cat['V_YEAR'] = df_train_cat['V_YEAR'].astype(CategoricalDtype(ordered=True))
df_train_cat['P_PSN'] = df_train_cat['P_PSN'].astype(CategoricalDtype(ordered=True))
df_train_cat['P_AGE'] = df_train_cat['P_AGE'].astype('int')
df_train_cat['P_ISEV'] = df_train_cat['P_ISEV'].astype('int')

In [ ]:
# convert to the correct type
df_test_cat['C_YEAR'] = df_test_cat['C_YEAR'].astype(CategoricalDtype(ordered=True))
df_test_cat['C_MNTH'] = df_test_cat['C_MNTH'].astype(CategoricalDtype(ordered=True))
df_test_cat['C_WDAY'] = df_test_cat['C_WDAY'].astype(CategoricalDtype(ordered=True))
df_test_cat['C_HOUR'] = df_test_cat['C_HOUR'].astype(CategoricalDtype(ordered=True))
df_test_cat['C_VEHS'] = df_test_cat['C_VEHS'].astype('int')
df_test_cat['V_YEAR'] = df_test_cat['V_YEAR'].astype(CategoricalDtype(ordered=True))
df_test_cat['P_PSN'] = df_test_cat['P_PSN'].astype(CategoricalDtype(ordered=True))
df_test_cat['P_AGE'] = df_test_cat['P_AGE'].astype('int')
df_test_cat['P_ISEV'] = df_test_cat['P_ISEV'].astype('int')
print(df_test_cat.info())

In [5]:
total_test_Rows = df_test_cat.index.size
print("Number of Rows in test data: {}".format(total_test_Rows))

total_train_Rows = df_train_cat.index.size
print("Number of Rows in train data: {}".format(total_train_Rows))

Number of Rows in test data: 1300968
Number of Rows in train data: 3534154


In [27]:
#Split between data and class for training
Y = df_train_cat[df_train_cat.columns[-1]]
X = df_train_cat[df_train_cat.columns[0:df_train_cat.columns.size -1]]

Y_test = df_test_cat[df_test_cat.columns[-1]]
X_test = df_test_cat[df_test_cat.columns[0:df_test_cat.columns.size -1]]

In [28]:
print(Y.unique())
print(Y.groupby(Y_train).size())
print(Y_test.unique())
print(Y_test.groupby(Y_test).size())

[1, 0]
Categories (2, int64): [1, 0]
P_ISEV
0    1767077
1    1767077
Name: P_ISEV, dtype: int64
[0, 1]
Categories (2, int64): [0, 1]
P_ISEV
0    543649
1    757319
Name: P_ISEV, dtype: int64


In [35]:
if (True):    
    seed = 101
    print("Logistic Regression: Start")
    t_start =  time.time()
    print()
    print("Model: Logistic Regression")
    print(time.asctime( time.localtime(t_start) ))
    print()
    print("Using Stratified Kfold: {}".format(nfold))
    kfold = model_selection.KFold(n_splits=nfold, random_state=seed)
    
    models = []
    
    X_ = np.array(X)
    Y_ = np.array(Y)
    
    for train_index, test_index in kfold.split(X_):
        X1_train, X1_test = X_[train_index], X_[test_index]
        y1_train, y1_test = Y_[train_index], Y_[test_index]
        
        lr = LogisticRegression(C=1, random_state=0, solver='saga', multi_class='ovr', 
                            verbose=verbose_level, n_jobs=10, max_iter=model_max_iter)
        print()
        print("Model: Logistic Regression")
        print(lr)
        print()
        print("Logistic Regression: Fit")
        lr.fit(X1_train, y1_train)

        #add model for list
        models.append(lr)

        print("Logistic Regression: Predict")
        y_pred = lr.predict(X1_test)

        print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X1_train, y1_train)))
        print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X1_test, y1_test)))

        # print the intercept (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
        print("Logistic Regression: Intercept")
        print(lr.intercept_)

        # print the coeficients (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
        print("Logistic Regression: Coefficients")
        print(lr.coef_)

        print("Logistic Regression: Confusion Matrix")
        cnf_matrix_lg = confusion_matrix(y1_test, y_pred)
        print(cnf_matrix_lg)
    
        print("Logistic Regression: Classification Report")
        print(classification_report(y1_test, y_pred))

        t_end =  time.time()
        print(time.asctime( time.localtime(t_end) ))
    
    # save model to file
    pickle.dump(models, open(file_lr, "wb"))
    print("Logistic Regression: End")

Logistic Regression: Start

Model: Logistic Regression
Tue Dec  4 02:41:40 2018

Using Stratified Kfold: 10
[ 353416  353417  353418 ... 3534151 3534152 3534153]

Model: Logistic Regression
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=10,
          penalty='l2', random_state=0, solver='saga', tol=0.0001,
          verbose=3, warm_start=False)

Logistic Regression: Fit
convergence after 13 epochs took 31 seconds
Logistic Regression: Predict


[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   30.6s finished


Accuracy of logistic regression classifier on train set: 0.62
Accuracy of logistic regression classifier on test set: 0.61
Logistic Regression: Intercept
[0.12181894]
Logistic Regression: Coefficients
[[ 0.00820767  0.00671875  0.01127662 -0.02271095 -0.32280624 -0.00311421
   0.00817163  0.02975961  0.09728152  0.25044309  0.04217313  0.07538374
  -0.12385502 -0.50908969  0.00815731 -0.32392867 -0.14224575  0.48835798]]
Logistic Regression: Confusion Matrix
[[102616  45461]
 [ 93558 111781]]
Logistic Regression: Classification Report
             precision    recall  f1-score   support

          0       0.52      0.69      0.60    148077
          1       0.71      0.54      0.62    205339

avg / total       0.63      0.61      0.61    353416

Tue Dec  4 02:42:17 2018
[      0       1       2 ... 3534151 3534152 3534153]

Model: Logistic Regression
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr

[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   34.5s finished


Accuracy of logistic regression classifier on train set: 0.62
Accuracy of logistic regression classifier on test set: 0.60
Logistic Regression: Intercept
[0.11974495]
Logistic Regression: Coefficients
[[ 0.00833139  0.00698844  0.01107335 -0.02211751 -0.32241109 -0.00327778
   0.00853769  0.02943044  0.09877754  0.25094203  0.04213656  0.07547784
  -0.12419451 -0.5085123   0.0081375  -0.32654973 -0.14239755  0.49018541]]
Logistic Regression: Confusion Matrix
[[102474  45049]
 [ 94899 110994]]
Logistic Regression: Classification Report
             precision    recall  f1-score   support

          0       0.52      0.69      0.59    147523
          1       0.71      0.54      0.61    205893

avg / total       0.63      0.60      0.61    353416

Tue Dec  4 02:42:57 2018
[      0       1       2 ... 3534151 3534152 3534153]

Model: Logistic Regression
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr

...

[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   33.6s finished


Accuracy of logistic regression classifier on train set: 0.62
Accuracy of logistic regression classifier on test set: 0.60
Logistic Regression: Intercept
[0.64469968]
Logistic Regression: Coefficients
[[ 0.00984901  0.00527804  0.00728325 -0.03353188 -0.3478647  -0.00255549
  -0.01731721  0.0248668   0.08912035  0.22316374  0.0412138   0.07730998
  -0.1515433  -0.56018893  0.00764423 -0.34092008 -0.146954    0.43949589]]
Logistic Regression: Confusion Matrix
[[136547  96118]
 [ 45827  74923]]
Logistic Regression: Classification Report
             precision    recall  f1-score   support

          0       0.75      0.59      0.66    232665
          1       0.44      0.62      0.51    120750

avg / total       0.64      0.60      0.61    353415

Tue Dec  4 02:47:30 2018
[      0       1       2 ... 3180736 3180737 3180738]

Model: Logistic Regression
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr

[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   34.1s finished


Accuracy of logistic regression classifier on train set: 0.63
Accuracy of logistic regression classifier on test set: 0.43
Logistic Regression: Intercept
[1.46889414]
Logistic Regression: Coefficients
[[ 0.01226555  0.00279343  0.00077514 -0.05247273 -0.387008   -0.00168901
  -0.05719052  0.0168505   0.07436339  0.17704345  0.03988152  0.0802104
  -0.19361064 -0.64661991  0.00672214 -0.36359441 -0.15166949  0.35896251]]
Logistic Regression: Confusion Matrix
[[150217 203198]
 [     0      0]]
Logistic Regression: Classification Report
             precision    recall  f1-score   support

          0       1.00      0.43      0.60    353415
          1       0.00      0.00      0.00         0

avg / total       1.00      0.43      0.60    353415

Tue Dec  4 02:48:09 2018
Logistic Regression: End


C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [20]:
if enable_lr:
    seed = 101
    print("Logistic Regression: Start")
    t_start =  time.time()
    print()
    print("Model: Logistic Regression")
    print(time.asctime( time.localtime(t_start) ))
    print()
    print("Using Kfold: {}".format(nfold))
    kfold = model_selection.KFold(n_splits=nfold, random_state=seed)
    
    lr = LogisticRegression(C=1, random_state=0, solver='saga', multi_class='ovr', 
                            verbose=verbose_level, n_jobs=10, max_iter=model_max_iter)
    print()
    print("Model: Logistic Regression")
    print(lr)
    print()
    print("Logistic Regression: Fit")
    results = model_selection.cross_val_score(lr, X_train, Y_train, cv=kfold)
    #lr.fit(X_train, Y_train)
    
    print(results.mean())
    print(results)
    
    # save model to file
    pickle.dump(lr, open(file_lr, "wb"))
    
    
if predict_lr:
    # load model from file
    loaded_model = pickle.load(open(file_lr, "rb"))
    print("Logistic Regression: Predict")
    y_pred = lr.predict(X_test)

    print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

    # print the intercept (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Intercept")
    print(lr.intercept_)

    # print the coeficients (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Coefficients")
    print(lr.coef_)

    print("Logistic Regression: Confusion Matrix")
    cnf_matrix_lg = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_lg)
    
    print("Logistic Regression: Classification Report")
    print(classification_report(Y_test, y_pred))

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
print("Logistic Regression: End")

Logistic Regression: Start

Model: Logistic Regression
Tue Dec  4 01:44:14 2018

Using Kfold: 10

Model: Logistic Regression
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=10,
          penalty='l2', random_state=0, solver='saga', tol=0.0001,
          verbose=3, warm_start=False)

Logistic Regression: Fit
convergence after 13 epochs took 31 seconds


[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   30.3s finished


convergence after 14 epochs took 34 seconds


[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   33.5s finished


convergence after 14 epochs took 33 seconds


...

convergence after 14 epochs took 33 seconds


[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   32.7s finished


convergence after 14 epochs took 32 seconds


[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   32.9s finished


0.5857633609003906
[0.60664203 0.6040134  0.6031221  0.60429635 0.60393022 0.60378309
 0.60428109 0.60415942 0.5983617  0.42504421]
Logistic Regression: End
